In [4]:
import os
os.chdir('..')
from inputGetter import get_input
import heapq

content = get_input(2024, 16)[:-1]

# Part A
An interesting graph problem using Dijkstra. However, the extra information of direction makes it more intricate. Had to use 3d array of `min_cost[r][c][direction]` to track on all cells and directions

In [5]:
content = content.split('\n')
sz_r, sz_c = len(content), len(content[0])
# Contains info on shortest path on all cells, and in all 4 directions
min_cost = [[[float('inf')] * 4 for _ in range(sz_c)] for _ in range(sz_r)]
min_cost[sz_r - 2][1][3] = 0

# Directions encoded as '^' = 0, 'v' = 1, '<' = 2, '>' = 3
directions = [(-1, 0, 0), (1, 0, 1), (0, -1, 2), (0, 1, 3)]

# Priority queue using heapq
# Put the current cost info, so the heap can prioritize based on cost, like how Dijkstra do
pq = []
heapq.heappush(pq, (0, sz_r - 2, 1, 3))

while pq:
    current_cost, r, c, direction = heapq.heappop(pq)

    for add_r, add_c, dir in directions:
        new_r, new_c = r + add_r, c + add_c
        if 0 <= new_r < sz_r and 0 <= new_c < sz_c and content[new_r][new_c] != '#':
            extra_cost = 1 if direction == dir else 1001

            if current_cost + extra_cost < min_cost[new_r][new_c][dir]:
                min_cost[new_r][new_c][dir] = current_cost + extra_cost
                heapq.heappush(pq, (min_cost[new_r][new_c][dir], new_r, new_c, dir))

res_id = 0
for i in range(4):
    if min_cost[1][sz_c - 2][i] <= min_cost[1][sz_c - 2][res_id]:
        res_id = i

print("Part A:", min_cost[1][sz_c - 2][res_id])

Part A: 143564


# Part B
The strat here is to track back from the goal cell and direction, to the beginning. If a cell with a specific direction lies on the optimal path, it must have the right cost like expected

In [6]:
'''
This has the requirements on each cell. It would decide the required cost of previous cell on the optimal path
For example, a cell in position that has distance (1, 0) from this cell -> It's on the bottom side of this cell
-> Top direction of this cell (which is encoded as 0) would has to have cost less than 1 to be on optimal path 
(Because no turn). Other directions only need to have cost less than 1001 to be optimal (1 move + 1 turn)
'''
cells_requirements = [(1, 0, 0), (-1, 0, 1), (0, 1, 2), (0, -1, 3)]
stack = []
visited = [[[False] * 4 for _ in range(sz_c)] for _ in range(sz_r)]

for i in range(4):
    if min_cost[1][sz_c - 2][i] == min_cost[1][sz_c - 2][res_id]:
        stack.append((1, sz_c - 2, i))
        visited[1][sz_c - 2][i] = True

while stack:
    r, c, direction = stack.pop()

    for add_r, add_c, required_direction in cells_requirements:
        new_r, new_c = r + add_r, c + add_c
        if 0 <= new_r < sz_r and 0 <= new_c < sz_c:
            for i in range(4):
                if not visited[new_r][new_c][i]:
                    req_cost = min_cost[r][c][direction] - (1 if i == required_direction else 1001)

                    if min_cost[new_r][new_c][i] == req_cost:
                        visited[new_r][new_c][i] = True
                        stack.append((new_r, new_c, i))

# If a cell has any direction that got into the optimal path, we take that cell
res_b = 0
for r in range(sz_r):
    for c in range(sz_c):
        res_b += any(visited[r][c][direction] for direction in range(4))

print("Part B:", res_b)

Part B: 593
